In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('allegro/herbert-base-cased')
bert_model = AutoModel.from_pretrained("allegro/herbert-base-cased")

Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.sso.sso_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.sso.sso_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
import pandas as pd
df = pd.read_excel('NLP_CLEAN.xlsx')

In [53]:
text = df[['nlp_2', 'nlp_3', 'nlp_4', 'nlp_5']].values
text = text.flatten()

2088

In [95]:
inputs = tokenizer(list(text), return_tensors='pt', max_length=512, truncation=True, padding='max_length') 
inputs['labels'] = inputs.input_ids.detach().clone()
inputs.input_ids[0]

tensor([    0,  3379,  1028,  2017,  2134, 23389,  1899,  3379,  1028, 37696,
         1899, 24656,  6585,  1899,  2954,  2025,    87,  3953,  2243,  1899,
        47890,  2483,  1899, 13186,  1011,  1899,  3248, 10321,  3304,  1899,
        30926,  4167,  1899,    57,  2009, 14323,  1899, 25503,  1899,     2,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1, 

In [92]:
rand = torch.rand(inputs.input_ids.shape)
rand

tensor([[0.0289, 0.0846, 0.4290,  ..., 0.4452, 0.1691, 0.1806],
        [0.2996, 0.8415, 0.2706,  ..., 0.2499, 0.2361, 0.7869],
        [0.1012, 0.2050, 0.7596,  ..., 0.2806, 0.9158, 0.7715],
        ...,
        [0.9598, 0.6893, 0.3946,  ..., 0.9117, 0.4985, 0.6031],
        [0.5516, 0.7210, 0.5911,  ..., 0.3245, 0.4975, 0.2062],
        [0.2084, 0.4292, 0.3041,  ..., 0.2938, 0.8050, 0.5884]])

In [93]:
mask_arr = (rand < 0.15) * (inputs.input_ids != 0) * (inputs.input_ids != 1) * (inputs.input_ids != 2)
mask_arr

tensor([[False,  True, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [97]:
torch.flatten(mask_arr[0].nonzero()).tolist()

[1, 3, 8, 9, 11, 20, 21, 25, 32]